## Scraper

Scrape webpages for DoB and DoD info.

Write compiled information to CSV

In [1]:
import pandas as pd


### Make a name list to scrape

In [2]:
nameList=["Kelvin"]

In [5]:
data=[]
for name in nameList:
    print(name)
    data.append({'name':name})
    data[-1]['born']=None
    data[-1]['died']=None

Kelvin


In [6]:
df_data=pd.DataFrame(data)
df_data

,name,born,died
0,Kelvin,None,None
